In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Id Dos times e Preparação da query

In [ ]:
import requests
import json as js
import pandas as pd
from time import sleep

endpoint = "teams/statistics"
url = f"https://v3.football.api-sports.io/{endpoint}"

teamsId = [127, 128, 121, 130, 134, 126, 119, 131, 151, 154, 133, 118, 124, 1062, 120, 129, 135, 150, 132, 145]
timeAtual = 130

querystring = {"league":"71","season":"2019","team":f"{timeAtual}"}

headers = {
  'x-rapidapi-key': REMOVIDO,
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

#Pega as estatisticas de cada clube da Serie A

In [ ]:
clubes_desempenho = pd.DataFrame(columns=['id','nome', 'maiorSequenciaEmpate','maiorSequenciaDerrota', 'maiorSequenciaVitoria', 'totalAmarelos', 'totalVermelhos', 'forma'])
clubes_golsStats = pd.DataFrame(columns=['id', 'nome', 'jogosSemTomarGol', 'jogosSemMarcarGol', 'maiorGolsContraFora', 'maiorGolsContraCasa', 'maiorGolsAfavorFora', 'maiorGolsAfavorCasa', 'mediaGolsContraFora', 'mediaGolsAfavorFora', 'mediaGolsContraCasa', 'mediaGolsAfavorCasa'])
formacoes = pd.DataFrame(columns=['idTime', 'nomeTime', 'formacao', 'vezesJogadas'])

In [ ]:
response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()
print(response.text)

{"get":"teams\/statistics","parameters":{"league":"71","season":"2019","team":"130"},"errors":[],"results":11,"paging":{"current":1,"total":1},"response":{"league":{"id":71,"name":"Serie A","country":"Brazil","logo":"https:\/\/media.api-sports.io\/football\/leagues\/71.png","flag":"https:\/\/media.api-sports.io\/flags\/br.svg","season":2019},"team":{"id":130,"name":"Gremio","logo":"https:\/\/media.api-sports.io\/football\/teams\/130.png"},"form":"LDLDLWLWWWDDDLDWDWWWWLDWWLLWWWWWLWLWWL","fixtures":{"played":{"home":19,"away":19,"total":38},"wins":{"home":12,"away":7,"total":19},"draws":{"home":3,"away":5,"total":8},"loses":{"home":4,"away":7,"total":11}},"goals":{"for":{"total":{"home":38,"away":26,"total":64},"average":{"home":"2.0","away":"1.4","total":"1.7"},"minute":{"0-15":{"total":10,"percentage":"15.15%"},"16-30":{"total":8,"percentage":"12.12%"},"31-45":{"total":12,"percentage":"18.18%"},"46-60":{"total":9,"percentage":"13.64%"},"61-75":{"total":11,"percentage":"16.67%"},"76-90"

In [ ]:
def somarCartoes(data, cor):
  soma = 0
  if data['response']['cards'][cor]['0-15']['total'] is not None:
    soma += data['response']['cards'][cor]['0-15']['total']
  if data['response']['cards'][cor]['16-30']['total'] is not None:
    soma += data['response']['cards'][cor]['16-30']['total']
  if data['response']['cards'][cor]['31-45']['total'] is not None:
    soma += data['response']['cards'][cor]['31-45']['total']
  if data['response']['cards'][cor]['46-60']['total'] is not None:
    soma += data['response']['cards'][cor]['46-60']['total']
  if data['response']['cards'][cor]['61-75']['total'] is not None:
    soma += data['response']['cards'][cor]['61-75']['total']
  if data['response']['cards'][cor]['76-90']['total'] is not None:
    soma += data['response']['cards'][cor]['76-90']['total']
  return soma

tabelaDesempenhoId = 0
tabelaFormacaoId = 0
for time in teamsId:
  querystring = {"league":"71","season":"2019","team":f"{time}"}
  response = requests.request("GET", url, headers=headers, params=querystring)
  data = response.json()

  clubeId = data['response']['team']['id']
  clubeNome = data['response']['team']['name']
  clubeMSE = data['response']['biggest']['streak']['draws']
  clubeMSD = data['response']['biggest']['streak']['loses']
  clubeMSV = data['response']['biggest']['streak']['wins']
  clubeAmarelos = somarCartoes(data, 'yellow')
  clubeVermelhos = somarCartoes(data, 'red')
  clubeForma = data['response']['form']

  clubeJSTG = data['response']['clean_sheet']['total']
  clubeJSMG = data['response']['failed_to_score']['total']

  clubeMGCF = data['response']['biggest']['goals']['against']['away']
  clubeMGCC = data['response']['biggest']['goals']['against']['home']
  clubeMGAF = data['response']['biggest']['goals']['for']['away']
  clubeMGAC = data['response']['biggest']['goals']['for']['home']

  clubeMeGCF = data['response']['goals']['against']['average']['away']
  clubeMeGCC = data['response']['goals']['against']['average']['home']
  clubeMeGAF = data['response']['goals']['for']['average']['away']
  clubeMeGAC = data['response']['goals']['for']['average']['home']

  clubes_desempenho.loc[str(tabelaDesempenhoId)] = [clubeId, clubeNome, clubeMSE, clubeMSD, clubeMSV, clubeAmarelos, clubeVermelhos, clubeForma]
  clubes_golsStats.loc[str(tabelaDesempenhoId)] = [clubeId, clubeNome, clubeJSTG, clubeJSMG, clubeMGCF, clubeMGCC, clubeMGAF, clubeMGAC, clubeMeGCF, clubeMeGCC, clubeMeGAF, clubeMeGAC]

  for i in range(len(data['response']['lineups'])):
    formacao = data['response']['lineups'][i]['formation']
    vezesJogadas = data['response']['lineups'][i]['played']
    formacoes.loc[str(tabelaFormacaoId)] = [clubeId, clubeNome, formacao, vezesJogadas]
    tabelaFormacaoId += 1
  tabelaDesempenhoId += 1
  sleep(10)

In [ ]:
formacoes

,idTime,nomeTime,formacao,vezesJogadas
0,127,Flamengo,4-2-3-1,25
1,127,Flamengo,4-1-4-1,3
2,127,Flamengo,4-4-2,3
3,127,Flamengo,4-2-2-2,3
4,127,Flamengo,4-1-3-2,2
...,...,...,...,...
95,145,Avai,5-3-2,2
96,145,Avai,4-5-1,2
97,145,Avai,5-4-1,2
98,145,Avai,4-4-2,2


In [ ]:
clubes_desempenho

,id,nome,maiorSequenciaEmpate,maiorSequenciaDerrota,maiorSequenciaVitoria,totalAmarelos,totalVermelhos,forma
0,127,Flamengo,1,1,8,90,3,WLDWLWWDWWDWLWWWWWWWWDWWWWWWDWWWDWWWWL
1,128,Santos,1,2,7,108,9,WWDWLDWWWWWWWLLDWDLLDWWWDWLDWWWWDWLWLW
2,121,Palmeiras,4,3,7,71,3,WDWWWWWWWDLDDDDLWWWWWDDLWWDWWWWDDLLLWW
3,130,Gremio,3,2,5,80,2,LDLDLWLWWWDDDLDWDWWWWLDWWLLWWWWWLWLWWL
4,134,Atletico Paranaense,1,2,4,82,1,WLDWLLWLLWWWLWLLWDLDWDWDLWDWDWDWWWWDWD
5,126,Sao Paulo,4,2,5,85,6,WWDWDLDDDDWWWWWLDLDWLDWDWLWWLWLLDDWLWW
6,119,Internacional,2,2,4,94,3,LWLWWDWLWLDWLDWLWWWWLDDLDWLWDLLWDDLWLW
7,131,Corinthians,2,2,3,69,1,LWDDWWDLWDWDWDWDWDLWWWDDLDLDLLWDDLWLWL
8,151,Goias,2,3,4,88,7,WLLWWLWWLDDLLLDWLLLWWWWLWDDLDWLLDWWLLW
9,154,Fortaleza EC,1,2,2,85,5,LWLLWDLLWWDLLWLDWLDLLWLWLLWDWDLWWDWWDW


In [ ]:
clubes_golsStats

,id,nome,jogosSemTomarGol,jogosSemMarcarGol,maiorGolsContraFora,maiorGolsContraCasa,maiorGolsAfavorFora,maiorGolsAfavorCasa,mediaGolsContraFora,mediaGolsAfavorFora,mediaGolsContraCasa,mediaGolsAfavorCasa
0,127,Flamengo,14,4,4,4,4,6,1.0,0.9,1.6,2.9
1,128,Santos,17,10,4,3,3,6,0.9,0.8,0.8,2.3
2,121,Palmeiras,15,4,3,3,2,6,1.0,0.7,1.1,2.1
3,130,Gremio,14,8,3,5,4,6,1.1,0.9,1.4,2.0
4,134,Atletico Paranaense,15,7,3,2,4,4,1.1,0.6,1.3,1.4
5,126,Sao Paulo,15,14,3,2,3,4,0.9,0.6,0.8,1.2
6,119,Internacional,12,11,3,2,3,3,1.3,0.8,0.8,1.5
7,131,Corinthians,17,10,4,2,3,3,1.1,0.7,0.9,1.3
8,151,Goias,10,9,6,3,2,4,2.2,1.2,0.8,1.6
9,154,Fortaleza EC,8,10,4,3,3,3,1.7,0.9,1.3,1.4


In [ ]:
#Salvar dataframe
clubes_golsStats.to_pickle('/content/drive/MyDrive/MC536 - Grupo GOLDB/DataFrames/estatisticasDeGol.pkl')
clubes_desempenho.to_pickle('/content/drive/MyDrive/MC536 - Grupo GOLDB/DataFrames/estatisticasDeDesempenho.pkl')
formacoes.to_pickle('/content/drive/MyDrive/MC536 - Grupo GOLDB/DataFrames/formacoesUsadas.pkl')

# Pegar IDs de cada partida (Fixture) do Brasileirão Série A 2019

In [ ]:
import http.client

conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': REMOVIDO
    }

conn.request("GET", "/v3/fixtures?league=71&season=2019", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"get":"fixtures","parameters":{"league":"71","season":"2019"},"errors":{"token":"Error\/Missing application key. Go to https:\/\/www.api-football.com\/documentation-v3 to learn how to get your API application key."},"results":0,"paging":{"current":1,"total":1},"response":[]}
